In [ ]:
# cian parser 
# try on jupiter not colab 

In [603]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

In [2]:
driver = webdriver.Chrome('C:\chromedriver')

In [31]:
def get_html(url, PARAMS = None): #give html code of url page
    html = driver.page_source
    return html

def get_content(html): #give BS html code
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_ = '_93444fe79c--container--2Kouc')
    return items
    
def get_urls():  #collect urls to flat from sotred pages 
    urls = []
    params = ['&sort=creation_date_desc', 
              '&sort=creation_date_asc', 
              '&sort=price_object_order', 
              '&sort=total_price_desc', 
              '&sort=street_name', 
              '&sort=walking_time']
    
    for par in tqdm(range(len(params))):
        param = params[par] 
        for i in range(1,55): 
            URL = driver.get(f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={i}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1{param}')
            html = get_html(URL)
            items = get_content(html)
            for item in items:
                urls.append(item.find('a', class_='_93444fe79c--link--39cNw').get('href'))
        urls = list(set(urls))
    return urls

def get_data(urls): #crutch for parse all falt params 
    prices, metro, floor, max_floor, size, msize, deadline, type_, bathroom, height, repairs, view, house = [], [], [], [], [], [], [], [], [], [], [], [], []
    garbage_chute, elevator, entrance, rooms, time = [], [], [], [], []
    for i in tqdm(range(len(urls))):
        url = urls[i]
        URL = driver.get(url)
        html = get_html(URL)
        soup = BeautifulSoup(html, 'html.parser')
        
        try: 
            price = soup.find('span', class_ = 'a10a3f92e9--price_value--1iPpd').find('span').get('content')
            prices.append(int(price.replace(' ', '').rstrip('₽')))
        except AttributeError: prices.append(np.nan)
            
        try: metro.append(soup.find('a', class_ = 'a10a3f92e9--underground_link--AzxRC').get_text())
        except AttributeError: metro.append(np.nan)
           
        try: time.append(int(soup.find('span', class_ = 'a10a3f92e9--underground_time--1fKft').get_text()[4:6]))
        except AttributeError: time.append(np.nan)
        except ValueError: time.append(np.nan)
         
        try: rooms.append(int(soup.find('h1', class_ = 'a10a3f92e9--title--2Widg').get_text()[:1]))
        except ValueError: rooms.append(np.nan)
            
        block1 = soup.find_all('div', class_ = 'a10a3f92e9--info-value--18c8R')
        titles1 = soup.find_all('div', class_ = 'a10a3f92e9--info-title--2bXM9')
        
        flag = [0,0,0,0]
        for i in range(len(titles1)):
            if titles1[i].get_text() == 'Общая':
                flag[0] = 1
                try: size.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: size.append(np.nan)
            elif titles1[i].get_text() == 'Жилая':
                flag[1] = 1
                try: msize.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: msize.append(np.nan)
            elif titles1[i].get_text() == 'Этаж':
                flag[2] = 1
                floors = block1[i].get_text().split(' из ')
                floor.append(int(floors[0]))
                max_floor.append(int(floors[1]))
            elif titles1[i].get_text() == 'Построен' or titles1[i].get_text() == 'Срок сдачи':
                flag[3] = 1
                deadline.append(int(block1[i].get_text()[-4:]))
        if flag[0] == 0:
            size.append(np.nan)
        if flag[1] == 0:
             msize.append(np.nan)
        if flag[2] == 0:
            floor.append(np.nan)
            max_floor.append(np.nan)
        if flag[3] == 0:
            deadline.append(np.nan)

        block2 = soup.find_all('span', class_ = 'a10a3f92e9--value--3Ftu5')
        titles2 = soup.find_all('span', class_ = 'a10a3f92e9--name--3bt8k')

        flag = [0,0,0,0]
        for i in range(len(titles2)):
            if titles2[i].get_text() == 'Тип жилья':
                flag[0] = 1
                type_.append(block2[i].get_text())
            elif titles2[i].get_text() == 'Высота потолков':
                flag[1] = 1
                height.append(float(block2[i].get_text().rstrip('м').replace(',','.')))
            elif titles2[i].get_text() == 'Санузел':
                flag[2] = 1
                bathroom.append(block2[i].get_text())
            elif titles2[i].get_text() == 'Вид из окон':
                flag[3] = 1
                view.append(block2[i].get_text())
        if flag[0] == 0:
            type_.append(np.nan)
        if flag[1] == 0:
             height.append(np.nan)
        if flag[2] == 0:
            bathroom.append(np.nan)
        if flag[3] == 0:
            view.append(np.nan)
        
    return prices, metro, size, msize, floor, max_floor, deadline, type_, height, bathroom, view, rooms, time

In [29]:
urls = get_urls()

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [33:41<00:00, 336.90s/it]


In [32]:
prices, metro, size, msize, floor, max_floor, deadline, type_, height, bathroom, view, rooms, time = get_data(urls)

100%|████████████████████████████████████████████████████████████████████████████| 8090/8090 [7:53:50<00:00,  3.51s/it]


In [777]:
# create pandas dataframe
df = pd.DataFrame({ 
    'Ссылка' : urls, 
    'Метро' : metro, 
    'Время до метро' : time,
    'Количество комнат' : rooms,
    'Общая площадь' : size, 
    'Жилая площадь' : msize, 
    'Этаж' : floor,
    'Этажность дома' : max_floor,
    'Срок сдачи' : deadline,
    'Тип жилья' : type_,
    'Высота потолков' : height,
    'Ванная' : bathroom,
    'Вид из окна' : view,
    'Цена' : prices
    })
df

,Ссылка,Метро,Время до метро,Количество комнат,Общая площадь,Жилая площадь,Этаж,Этажность дома,Срок сдачи,Тип жилья,Высота потолков,Ванная,Вид из окна,Цена
0,https://www.cian.ru/sale/flat/250039498/,Международная,3.0,NaN,454.00,NaN,94,95,2017.0,Вторичка Апартаменты,NaN,"1 совмещенный, 1 раздельный",На улицу,490000000
1,https://www.cian.ru/sale/flat/256280788/,Щукинская,5.0,NaN,560.00,325.0,4,31,2008.0,Вторичка,NaN,NaN,NaN,243300525
2,https://www.cian.ru/sale/flat/249890041/,Семеновская,3.0,1.0,35.40,20.0,2,9,1971.0,Вторичка,2.64,1 совмещенный,Во двор,7997000
3,https://www.cian.ru/sale/flat/256429417/,Мичуринский проспект,34.0,2.0,44.50,NaN,2,28,2022.0,Новостройка,NaN,1 совмещенный,NaN,11950000
4,https://www.cian.ru/sale/flat/256459729/,Мичуринский проспект,10.0,1.0,48.00,19.9,18,25,2011.0,Вторичка,2.70,1 совмещенный,На улицу,16500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8085,https://www.cian.ru/sale/flat/256470500/,Окская,6.0,3.0,60.90,38.0,7,12,NaN,Вторичка,2.60,1 раздельный,Во двор,12500000
8086,https://www.cian.ru/sale/flat/256412791/,Юго-Западная,10.0,1.0,54.35,22.6,5,13,NaN,Вторичка,3.00,1 совмещенный,Во двор,19257600
8087,https://www.cian.ru/sale/flat/249240206/,Водный стадион,6.0,NaN,535.00,NaN,5,6,2008.0,Вторичка Пентхаус,3.20,4 совмещенных,На улицу и двор,274308010
8088,https://www.cian.ru/sale/flat/254454232/,Стахановская,6.0,1.0,36.40,11.5,3,9,2020.0,Вторичка,3.10,1 совмещенный,Во двор,10300000


In [787]:
from joblib import load
clf = load('rfr.joblib') 


In [789]:
clf.predict(123)

ValueError: Expected 2D array, got scalar array instead:
array=123.0.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.